In [3]:
from dengo.chemical_network import ChemicalNetwork, reaction_registry, cooling_registry, species_registry
import dengo.primordial_rates, dengo.primordial_cooling
from dengo.chemistry_constants import tiny, kboltz, mh
import numpy as np
import matplotlib.pyplot as plt
import sympy

In [4]:
dengo.primordial_rates.setup_primordial();

In [5]:
primordial = ChemicalNetwork()

In [6]:
for i in range(23):
    n = "k%02i" % (i+1)
    if n in reaction_registry:
        primordial.add_reaction(n)

Adding reaction: k01 : 1*H_1 + 1*de => 1*H_2 + 2*de
Adding reaction: k02 : 1*H_2 + 1*de => 1*H_1
Adding reaction: k03 : 1*He_1 + 1*de => 1*He_2 + 2*de
Adding reaction: k04 : 1*He_2 + 1*de => 1*He_1
Adding reaction: k05 : 1*He_2 + 1*de => 1*He_3 + 2*de
Adding reaction: k06 : 1*He_3 + 1*de => 1*He_2
Adding reaction: k07 : 1*H_1 + 1*de => 1*H_m0
Adding reaction: k08 : 1*H_m0 + 1*H_1 => 1*H2_1 + 1*de
Adding reaction: k09 : 1*H_1 + 1*H_2 => 1*H2_2
Adding reaction: k10 : 1*H2_2 + 1*H_1 => 1*H2_1 + 1*H_2
Adding reaction: k11 : 1*H2_1 + 1*H_2 => 1*H2_2 + 1*H_1
Adding reaction: k12 : 1*H2_1 + 1*de => 2*H_2 + 1*de
Adding reaction: k13 : 1*H2_1 + 1*H_1 => 3*H_1
Adding reaction: k14 : 1*H_m0 + 1*de => 1*H_1 + 2*de
Adding reaction: k15 : 1*H_m0 + 1*H_1 => 2*H_1 + 1*de
Adding reaction: k16 : 1*H_m0 + 1*H_2 => 2*H_1
Adding reaction: k17 : 1*H_m0 + 1*H_2 => 1*H2_2 + 1*de
Adding reaction: k18 : 1*H2_2 + 1*de => 2*H_1
Adding reaction: k19 : 1*H2_2 + 1*H_m0 => 1*H_1 + 1*H2_1
Adding reaction: k21 : 2*H_1 

In [7]:
# Initializing primordial network
NCELLS = 1
density = 1e7
temperature = np.logspace(2, 4, NCELLS)
temperature[:] = 2e3
X = 0.5

primordial.init_temperature((1e0, 1e8))

init_array = np.ones(NCELLS) * density
init_values = dict()
init_values['H_1']   = init_array * 0
init_values['H_2']   = X * init_array
init_values['H_m0']  = init_array * tiny
init_values['He_1']  = init_array * tiny
init_values['He_2']  = init_array * tiny
init_values['He_3']  = init_array * tiny
init_values['H2_1']  = init_array * tiny
init_values['H2_2']  = init_array * tiny
init_values['de']    = init_array * 0.0

total_density = primordial.calculate_total_density(init_values)
init_values["H_1"] = init_array.copy() - total_density
init_values = primordial.convert_to_mass_density(init_values)
init_values['de'] = primordial.calculate_free_electrons(init_values)
init_values['density'] = primordial.calculate_total_density(init_values)
number_density = primordial.calculate_number_density(init_values)

# set up initial temperatures values used to define ge
init_values['T'] = temperature

# calculate ge (very crudely, no H2 help here)
gamma = 5.0/3.0
init_values['ge'] = ((temperature * number_density * kboltz)
                     / (init_values['density'] * mh * (gamma - 1)))

In [8]:
# this gives you the rhs function of that particular species
# in this case, its H2_1, molecular H2
# f_H2 is an sympy object!
f_H2 = primordial.species_total('H2_1')

# d f_H2 / d H_2 the jacobian component
dfH2_dH2 = f_H2.diff('H2_1')
dfH2_dH2

-k11[i]*H_2 - k12[i]*de - k13[i]*H_1 + k21[i]*H_1**2 - 2*k23[i]*H2_1

In [9]:
rxn_rates = sorted(primordial.reactions.keys())
species = sorted(primordial.species_list())

In [10]:
# The following gives the array of k01 rates at the temperatures
k01 = primordial.reactions['k01']
k01.coeff_fn(primordial)

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         9.17839771e-09,   9.10254611e-09,   9.02719165e-09])

In [12]:
def get_jac_component(species1, species2):
    f_species1 = primordial.species_total(species1)
    df1_f2 = f_species1.diff(species2)
    return df1_f2

In [13]:
# These are the arguments to the functions of the jacobian that can compute the numerical
# values of each jacobian component
species+rxn_rates+["i"]

['H2_1',
 'H2_2',
 'H_1',
 'H_2',
 'H_m0',
 'He_1',
 'He_2',
 'He_3',
 'de',
 'ge',
 'k01',
 'k02',
 'k03',
 'k04',
 'k05',
 'k06',
 'k07',
 'k08',
 'k09',
 'k10',
 'k11',
 'k12',
 'k13',
 'k14',
 'k15',
 'k16',
 'k17',
 'k18',
 'k19',
 'k21',
 'k22',
 'k23',
 'i']

In [22]:
# rxn_jac is the matrix of functions for the jacobian terms
# species_rates_list is a list of species values and list of rxn_rate arrays
# temp index is the index for the rxn_rate arrays corresponding to a specific temperature
def compute_jacobian(rxn_jac,species_rates_list,temp_index):
    jac = np.zeros((len(rxn_jac),len(rxn_jac)),dtype=float)
    for j in range(len(jac)):
        for k in range(len(jac)):
            jac[j][k] = rxn_jac[j][k](*species_rates_list,i=temp_index)
    return jac

In [27]:
def grad_descent(network, init_values, temp_indices, epsilon):
    # compute jacobian
    rxn_rates = sorted(network.reactions.keys())
    species = sorted(network.species_list())
    f_arguments = species+rxn_rates+['i']
    rxn_jac = np.zeros((len(species),len(species)),dtype=object)
    for j in range(len(species)):
        for k in range(len(species)):
            jac_fn = get_jac_component(species[j],species[k])
            rxn_jac[j][k] = sympy.lambdify(f_arguments,jac_fn,"numpy")

    species_values = []
    for s in species:
        species_values.append(init_values[s])
    rate_arrays = []
    for k in range(len(rxn_rates)):
        rate = primordial.reactions[rxn_rates[k]]
        rate_arrays.append(rate.coeff_fn(primordial))
    
    jac_argument = species_values+rate_arrays
    curr_jac = compute_jacobian(rxn_jac,jac_argument,temp_indices)

    sp_diff = epsilon + 1
    curr_sp = np.array(species_values)[:,0]
    
    learn_rate = .1
    
    #start descent here, updating species values, minimizing sum(jacobian)
    while(learn_rate < 1):
        
        #GD update step
        gradient = np.sum(curr_jac, axis=1)
        curr_sp = curr_sp - learn_rate*gradient
        
        #calculating new jacobian
        jac_argument = list(curr_sp)+rate_arrays
        curr_jac = compute_jacobian(rxn_jac,jac_argument,temp_indices)
        new_gradient = np.sum(curr_jac, axis=1)
        
        #calculating average error in s and s+1 jacobian species sums
        err_diff = np.mean(gradient-new_gradient)
        
        #updating learning rate using bold driver technique
        if(err_diff < 0):
            learn_rate *= 1.05
        elif(err_diff > 0):
            learn_rate *= .50
        else:
            break
    
    return(dict(zip(species,curr_sp)))

In [28]:
init_values

{'H2_1': array([  2.00000000e-23]),
 'H2_2': array([  2.00000000e-23]),
 'H_1': array([ 5039700.]),
 'H_2': array([ 5039700.]),
 'H_m0': array([  1.00794000e-23]),
 'He_1': array([  4.00260200e-23]),
 'He_2': array([  4.00260200e-23]),
 'He_3': array([  4.00260200e-23]),
 'T': array([ 2000.]),
 'de': array([ 5000000.]),
 'density': array([ 10079400.]),
 'ge': array([  2.45951338e+11])}

In [31]:
sp_values = init_values
temp_indices = 500
epsilon = .1

In [32]:
grad_descent(primordial, sp_values, temp_indices, epsilon)

('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species difference: ', 1.1)
('average updated species differen

{'H2_1': -0.2241674367474755,
 'H2_2': 0.16300731227797402,
 'H_1': 5039697.8137380444,
 'H_2': 5039700.7054573549,
 'H_m0': 1.6031248485742393,
 'He_1': 4.0026020002032014e-23,
 'He_2': -0.00022272709841250873,
 'He_3': 0.00022272709841250873,
 'de': 4999999.2655658722,
 'ge': 245951337993.10117}